# Torch preprocessors

Some preprocessors that don't act in the "draw action" method of agents based on numpy arrays, but directly in the fit or predict methods of agents based on PyTorch models.

In [ ]:
#| default_exp torch_utils.preprocessors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
from typing import Optional

import torch


TODO:
 - write descriptions
 - specify typings
 - ensure documentation on types, outputs, variable descriptions

In [ ]:
#| export

class FlattenTimeDim():
    """
    Preprocessor to flatten the time and feature dimension of the input.
    Used, e.g., to convert time-series data for models that cannot process
    a time dimension such as MLPs or Regression models.
    """

    def __init__(self, allow_2d: Optional[bool] = False):
        self.allow_2d = allow_2d

    def check_input(self, input: torch.Tensor):
        """
        Check that the input is a PyTorch tensor with the correct shape.

        Parameters:
        input (torch.Tensor): The input tensor to check.

        Raises:
        TypeError: If the input is not a PyTorch tensor.
        ValueError: If the input tensor does not have the expected dimensions.
        """
        # Check if the input is a PyTorch tensor
        if not isinstance(input, torch.Tensor):
            raise TypeError(f"Expected input to be a torch.Tensor but got {type(input)} instead.")

        # Check if the input tensor has the correct dimensions
        if input.dim() == 3:
            # If the input is 3D, it is valid regardless of allow_2d
            return
        elif input.dim() == 2:
            # If the input is 2D, check if 2D is allowed
            if not self.allow_2d:
                raise ValueError(
                    f"Expected input to have 3 dimensions with shape (batch_size, timesteps, features), "
                    f"but got 2D shape {input.shape} instead. 2D inputs are not allowed when allow_2d is False."
                )
        else:
            # If the input has dimensions other than 2 or 3
            expected_shape_msg = (
                "Expected input to have 3 dimensions with shape (batch_size, timesteps, features)"
                if not self.allow_2d
                else "Expected input to have 3 dimensions with shape (batch_size, timesteps, features) "
                     "or 2 dimensions with shape (batch_size, features)"
            )
            raise ValueError(f"{expected_shape_msg}, but got shape {input.shape} instead.")


    def __call__(self, input: torch.Tensor) -> torch.Tensor:
        """
        Process the input tensor by keeping the batch dimension and flattening
        the time and feature dimensions.

        Args:
            input (torch.Tensor): The input tensor with shape (batch_size, timesteps, features).

        Returns:
            torch.Tensor: The reshaped output tensor with shape (batch_size, timesteps * features).
        """

        # Validate the input tensor
        self.check_input(input)

        if input.dim()==2:
            output = input
        else:
            # Keep the batch dimension, flatten time and feature dimensions
            batch_size, timesteps, features = input.shape
            output = input.view(batch_size, -1)

        return output

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()